# Geocoding


### Table of Content

1. [Import Libraries and the dataset](#import)
2. 

#### 1. Import Libraries and the dataset. <a id = 'import'></a>

In [1]:
#Libraies imported
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import re
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
#Dataset imported

updated_file = '../Datasets/lat_lon_updated_for_real_estate_with_remarks.csv'
real_estate = pd.read_csv(f'../Datasets/{updated_file}')

#Reset Index
real_estate.reset_index(drop=True)

/var/folders/kv/4xjj0g696_lcyc7ms5pdd2t00000gn/T/ipykernel_3741/465869204.py:4: DtypeWarning: Columns (10,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate = pd.read_csv(f'../Datasets/{updated_file}')


,Unnamed: 0.1,Unnamed: 0,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,...,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Full Address,latitude,longitude,month_recorded,year_recorded
0,0,0,2020180,2020,2021-03-01,Berlin,1539 FARMINGTON AVE,234200.0,130000.0,1.801500,...,Two Family,8.0,NaN,NaN,NaN,"1539, Farmington Ave, Berlin, Connecticut, Uni...",41.622355,-72.746275,3,2021
1,1,1,200086,2020,2021-08-10,Bethlehem,39 WOODLAND RD,168900.0,352000.0,0.479800,...,Single Family,NaN,NaN,NaN,POINT (-73.18197 41.64672),"39, Woodland Rd, Bethlehem, Connecticut, Unite...",-73.181970,41.646720,8,2021
2,2,2,200258,2020,2021-01-22,Branford,34 SUNRISE COVE CAMP,530500.0,700000.0,0.757800,...,Single Family,NaN,WATERFRONT/COTTAGE ONLY/LAND IS ASSOCIATION OWNED,NaN,NaN,"34, Sunrise Cove Camp, Branford, Connecticut, ...",0.000000,0.000000,1,2021
3,3,3,201212,2020,2021-08-23,Bristol,8 JENNINGS TERR,88060.0,91000.0,0.967600,...,Single Family,14.0,FORECLOSURE SALE OUT OF BANK,NaN,NaN,"8, Jennings Terr, Bristol, Connecticut, United...",41.678618,-72.952319,8,2021
4,4,4,2000330,2020,2021-05-20,Cheshire,844 HIGHLAND AVE,144220.0,204000.0,0.706900,...,Single Family,25.0,DEFERRED MAINTENANCE,NaN,NaN,"844, Highland Ave, Cheshire, Connecticut, Unit...",41.521906,-72.896793,5,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461888,461888,461902,190105,2019,2020-02-25,Plymouth,18 OVERLOOK RD,104130.0,80000.0,1.301625,...,Single Family,14.0,NaN,BANK SALE PER MLS,POINT (-72.98492 41.64753),"18, Overlook Rd, Plymouth, Connecticut, United...",-72.984920,41.647530,2,2020
461889,461889,461903,19921,2019,2019-11-18,West Haven,75 CLOVER ST,125230.0,246000.0,0.509100,...,Single Family,NaN,NaN,NaN,POINT (-72.96445 41.25722),"75, Clover St, West Haven, Connecticut, United...",-72.964450,41.257220,11,2019
461890,461890,461904,190272,2019,2020-08-03,Wilton,145 WHIPSTICK RD,681870.0,1134708.0,0.600900,...,Single Family,25.0,NaN,"PER MLS CLOSING PRICE = $1,145,000",NaN,"145, Whipstick Rd, Wilton, Connecticut, United...",41.245241,-73.465908,8,2020
461891,461891,461905,190272,2019,2020-06-24,New London,4 BISHOP CT,60410.0,53100.0,1.137665,...,Single Family,14.0,NaN,NaN,NaN,"4, Bishop Ct, New London, Connecticut, United ...",41.353418,-72.104892,6,2020


The dataset is imported. 

Because the dataset is continually being updated after retrieving the coordinates from Open Street Map API, the  latest version of the dataset should be imported, not the raw dataset. 

The latest updated dataset can be downloaded here:

In [3]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461893 entries, 0 to 461892
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0.1      461893 non-null  int64  
 1   Unnamed: 0        461893 non-null  int64  
 2   Serial Number     461893 non-null  int64  
 3   List Year         461893 non-null  int64  
 4   Date Recorded     461893 non-null  object 
 5   Town              461893 non-null  object 
 6   Address           461845 non-null  object 
 7   Assessed Value    461893 non-null  float64
 8   Sale Amount       461893 non-null  float64
 9   Sales Ratio       461893 non-null  float64
 10  Property Type     300553 non-null  object 
 11  Residential Type  296116 non-null  object 
 12  Non Use Code      245326 non-null  float64
 13  Assessor Remarks  149858 non-null  object 
 14  OPM remarks       9916 non-null    object 
 15  Location          197691 non-null  object 
 16  Full Address      44

In [9]:
def save_to_csv():
    real_estate.to_csv('../Datasets/lat_lon_updated_for_real_estate_with_remarks.csv')

In [5]:

# Initialize the geocoder
geolocator = Nominatim(user_agent="bootcamp student")

# Assuming real_estate is defined elsewhere in your code
real_estate = pd.DataFrame(real_estate)

max_retries = 1
last_successful_index = None  # Initialize to None
save_interval = 500  # Define the interval for saving the CSV file
processed_count = 0  # Initialize the counter
print_interval = 100

start_index = 189875
retries = 0 


In [6]:

for index, row in real_estate.iloc[start_index:].iterrows():
    processed_count += 1
    if processed_count % print_interval == 0:
        print(index, "th number of row is being worked on")
    if processed_count % save_interval == 0:
        print("Saving the updated .csv file")
        save_to_csv()
    if pd.notnull(row["longitude"]) or pd.notnull(row["longitude"]):
        #print(index, "th row has lat/lon values")
        # Skip to the next row if either Latitude and Longitude are already present
        continue
    address = row["Full Address"]
    retries =0 
    while retries < max_retries:
        try:
            retries +=1
            #print(index, "th row begin geocoding")
            location = geolocator.geocode(address, timeout=7)
            if location is not None:
                if location.latitude and location.longitude:
                    real_estate.at[index, "latitude"] = location.latitude
                    real_estate.at[index, "longitude"] = location.longitude
                    last_successful_index = index  # Update the index of the last successful geocode
                    #print(index, "geocoded")
                else:
                    real_estate.at[index, "latitude"] = 0
                    real_estate.at[index, "longitude"] = 0
                break  # Successful geocode, break out of the loop

        except GeocoderTimedOut:
            print("Geocoding timed out for:", address)
            print("Retrying... (Attempt", retries, "of", max_retries, ")")
            
    else:
        real_estate.at[index, "latitude"] = 0
        real_estate.at[index, "longitude"] = 0

187575 th number of row is being worked on
187675 th number of row is being worked on
187775 th number of row is being worked on
187875 th number of row is being worked on
187975 th number of row is being worked on
Saving the updated .csv file
188075 th number of row is being worked on
188175 th number of row is being worked on
188275 th number of row is being worked on
188375 th number of row is being worked on
188475 th number of row is being worked on
Saving the updated .csv file
188575 th number of row is being worked on
188675 th number of row is being worked on
188775 th number of row is being worked on
188875 th number of row is being worked on
188975 th number of row is being worked on
Saving the updated .csv file
189075 th number of row is being worked on
189175 th number of row is being worked on
189275 th number of row is being worked on
189375 th number of row is being worked on
189475 th number of row is being worked on
Saving the updated .csv file
189575 th number of row 

KeyboardInterrupt: 

In [10]:
real_estate = real_estate.loc[:, ~real_estate.columns.str.contains('Unnamed')]

In [11]:

# Save the final DataFrame to a CSV file
save_to_csv()
